<a href="https://colab.research.google.com/github/label-complex/ECG_prediction_PTB_XL/blob/main/Feature_Analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Data and Libs
The whole database is saved in google drive. Also, we need a specific library WFDB to read our ECG datas. The detailed usage of this library will be described in following parts.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wfdb wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=50ff58dbff3a99d4df0560c15a3f01a134145f49fbfdbc49c9c4741616b9f13a
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: SoundFile
    Found existing installation: soundfile 0.12.1
    Uninstalling soundfile-0.12.1:
      Successfully uninstalled soundfile-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.


In [ ]:
import os
import numpy as np, sys,os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
import wfdb
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
#from keras.preprocessing.sequence import pad_sequences
import math
import warnings

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

def load_challenge_data(filename):
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)
    new_file = filename.replace('.mat','.hea')
    input_header_file = os.path.join(new_file)
    with open(input_header_file,'r') as f:
        header_data=f.readlines()
    return data, header_data

path = 'drive/MyDrive/ptb-xl-EGC-dataset-1.0.3/records100/00000/'
sampling_rate=100

# load and convert annotation data
#Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
#Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
#X = load_raw_data(Y, sampling_rate, path)

# Database Description

>The [PTB-XL ECG dataset](https://physionet.org/content/ptb-xl/1.0.3/) is a large dataset of 21799 clinical 12-lead ECGs from 18869 patients of 10 second length, where 52% are male and 48% are female with ages covering the whole range from 0 to 95 years (median 62 and interquantile range of 22). The raw waveform data was annotated by up to two cardiologists, who assigned potentially multiple ECG statements to each record. The in total 71 different [ECG statements](https://physionet.org/content/ptb-xl/1.0.3/scp_statements.csv) conform to the SCP-ECG standard and cover diagnostic, form, and rhythm statements. The dataset is complemented by extensive metadata on demographics, infarction characteristics, likelihoods for diagnostic ECG statements as well as annotated signal properties.

##File Structure
These records are divided into 21 folders. For each record, there ar two file types:

1. Header files (.hea)
2. Signal files (.mat)

The waveform files are stored in WaveForm DataBase ([WFDB](https://wfdb.readthedocs.io/en/latest/)) format with 16 bit precision at a resolution of 1μV/LSB and a sampling frequency of 500Hz (*/records500*). For the user’s convenience we also release a downsampled versions of the waveform data at a sampling frequency of 100Hz (*records100/*).

In general, the dataset is organized as follows:

<dl>
<dt>ptbxl
<dt>├── ptbxl_database.csv</dt>
<dt>├── scp_statements.csv</dt>
<dt>├── /records100</dt>
<dt>└── /── /00000</dt>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 00001_lr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 00001_lr.hea</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 00999_lr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── 00999_lr.hea</dd>
<dt>└── /── /...</dt>
<dt>└── /── /21000</dt>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 21001_lr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 21001_lr.hea</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 21837_lr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── 21837_lr.hea</dd>
<dt>├── /records500</dt>
<dt>└── /── /00000</dt>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 00001_hr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 00001_hr.hea</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 00999_hr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── 00999_hr.hea</dd>
<dt>└── /── /...</dt>
<dt>└── /── /21000</dt>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 21001_hr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 21001_hr.hea</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── 21837_hr.dat</dd>
<dd>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── 21837_hr.hea</dd>
</dl>


The distribution of diagnosis is as follows, where we restrict for simplicity to diagnostic statements aggregated into superclasses (note: sum of statements exceeds the number of records because of potentially multiple labels per record):
<div align = "center">
  <table>
    <thead>
      <tr>
        <th scope="col">#Records</th>
        <th scope="col">Superclass</th>
        <th scope="col">Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>9514</td>
        <td>NORM</td>
        <td>Normal ECG</td>
      </tr>
      <tr>
        <td>5469</td>
        <td>MI</td>
        <td>Myocardial Infarction</td>
      </tr>
      <tr>
        <td>5235</td>
        <td>STTC</td>
        <td>ST/T Change</td>
      </tr>
      <tr>
        <td>4898</td>
        <td>CD</td>
        <td>Conduction Disturbance</td>
      </tr>
      <tr>
        <td>2649</td>
        <td>HYP</td>
        <td>Hypertrophy</td>
      </tr>
    </tbody>
  </table>
</div>



All relevant metadata is stored in [ptbxl_database.csv](https://physionet.org/content/ptb-xl/1.0.3/ptbxl_database.csv) with one row per record identified by ecg_id. It contains 28 columns that can be categorized into:

1. Identifiers: Each record is identified by a unique ecg_id. The corresponding patient is encoded via patient_id. The paths to the original record (500 Hz) and a downsampled version of the record (100 Hz) are stored in *filename_hr* and *filename_lr*.
2. General Metadata: demographic and recording metadata such as *age, sex, height, weight, nurse, site, device* and *recording_date*
3. [ECG statements](https://physionet.org/content/ptb-xl/1.0.3/scp_statements.csv): core components are scp_codes (SCP-ECG statements as a dictionary with entries of the form statement: likelihood, where likelihood is set to 0 if unknown) and report (report string). Additional fields are *heart_axis, infarction_stadium1, infarction_stadium2, validated_by, second_opinion, initial_autogenerated_report* and *validated_by_human*.
4. Signal Metadata: signal quality such as noise (static_noise and burst_noise), baseline drifts (baseline_drift) and other artifacts such as electrodes_problems. We also provide extra_beats for counting extra systoles and pacemaker for signal patterns indicating an active pacemaker.
5. Cross-validation Folds: recommended 10-fold train-test splits (strat_fold) obtained via stratified sampling while respecting patient assignments, i.e. all records of a particular patient were assigned to the same fold. Records in fold 9 and 10 underwent at least one human evaluation and are therefore of a particularly high label quality. It is proposed to use folds 1-8 as training set, fold 9 as validation set and fold 10 as test set.

In [ ]:
ex_id = '00943_lr'
ex_data = wfdb.rdsamp(path+ex_id)
print(ex_data)

(array([[-0.074, -0.253, -0.179, ..., -0.265, -0.091, -0.132],
       [-0.074, -0.25 , -0.176, ..., -0.271, -0.096, -0.135],
       [-0.064, -0.235, -0.171, ..., -0.273, -0.101, -0.138],
       ...,
       [ 0.073, -0.051, -0.123, ..., -0.546, -0.276,  0.074],
       [ 0.03 , -0.027, -0.057, ..., -0.309, -0.169,  0.086],
       [ 0.048,  0.054,  0.006, ..., -0.134, -0.039,  0.19 ]]), {'fs': 100, 'sig_len': 1000, 'n_sig': 12, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV'], 'sig_name': ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'], 'comments': []})
